In [1]:
import os
import cv2
import numpy as np
from ultralytics import YOLO
from tqdm import tqdm

# --- KONFIGURASI (WAJIB DISESUAIKAN) ---

# 1. Path ke model yang khusus segmentasi DAUN
LEAF_MODEL_PATH = '../results/maskDaun/runs/train_maskDaun_m/weights/best.pt'

# 2. Path ke model yang khusus segmentasi LESI/PENYAKIT
DISEASE_MODEL_PATH = '../results/maskLesi/runs/train_maskLesi_m/weights/best.pt'

# 3. Path ke FOLDER berisi gambar yang akan dianalisis
IMAGE_FOLDER = '../dataset/maskLesi/valid/images'

# 4. Folder untuk menyimpan semua hasil visualisasi gabungan
OUTPUT_FOLDER = '../view/maskLesi/'

# --- AKHIR KONFIGURASI ---


def find_and_crop_largest_leaf(leaf_model, image_path):
    """
    Mendeteksi semua daun, menemukan yang terbesar berdasarkan bounding box,
    dan meng-crop gambar asli pada area tersebut.
    """
    # Jalankan prediksi daun
    results = leaf_model(image_path, verbose=False)
    boxes = results[0].boxes
    
    if boxes is None or len(boxes) == 0:
        return None # Tidak ada daun terdeteksi

    largest_area = 0
    largest_box = None

    # Cari bounding box dengan area terluas
    for box in boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        area = (x2 - x1) * (y2 - y1)
        if area > largest_area:
            largest_area = area
            largest_box = (x1, y1, x2, y2)
    
    # Crop gambar asli berdasarkan box terbesar
    original_img = cv2.imread(image_path)
    x1, y1, x2, y2 = largest_box
    cropped_image = original_img[y1:y2, x1:x2]
    
    return cropped_image


def analyze_cropped_image(leaf_model, disease_model, cropped_image):
    """
    Menghitung severity pada gambar yang sudah di-crop.
    """
    # Prediksi pada gambar yang sudah di-crop
    leaf_results = leaf_model(cropped_image, verbose=False)
    disease_results = disease_model(cropped_image, verbose=False)

    leaf_masks = leaf_results[0].masks
    disease_masks = disease_results[0].masks

    total_leaf_pixels = sum(np.count_nonzero(m.data[0].cpu().numpy()) for m in leaf_masks) if leaf_masks else 0
    total_disease_pixels = sum(np.count_nonzero(m.data[0].cpu().numpy()) for m in disease_masks) if disease_masks else 0

    if total_leaf_pixels > 0:
        severity_percentage = (total_disease_pixels / total_leaf_pixels) * 100
    else:
        severity_percentage = 0

    return severity_percentage, leaf_masks, disease_masks


def create_combined_visualization(cropped_original_img, leaf_masks, disease_masks, severity):
    """
    Membuat tiga panel visualisasi dari gambar yang sudah di-crop.
    """
    # --- Panel 1: Gambar Asli (Sudah di-crop) ---
    panel1 = cropped_original_img.copy()
    cv2.putText(panel1, "1. Daun Terbesar", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)

    # --- Panel 2: Segmentasi Daun + Lesi ---
    panel2 = cropped_original_img.copy()
    cv2.putText(panel2, "2. Segmentasi", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)
    
    leaf_color = (0, 255, 0)
    disease_color = (0, 0, 255)
    
    if leaf_masks:
        for mask in leaf_masks:
            points = mask.xy[0].astype(np.int32)
            cv2.fillPoly(panel2, [points], leaf_color, lineType=cv2.LINE_AA)

    if disease_masks:
        for mask in disease_masks:
            points = mask.xy[0].astype(np.int32)
            cv2.fillPoly(panel2, [points], disease_color, lineType=cv2.LINE_AA)
            
    # --- Panel 3: Visualisasi Severity ---
    panel3 = cropped_original_img.copy()
    cv2.putText(panel3, "3. Analisis Severity", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)
    
    severity_text = f"Severity: {severity:.2f}%"
    font = cv2.FONT_HERSHEY_TRIPLEX
    font_scale = 1.2
    thickness = 2
    (text_width, text_height), _ = cv2.getTextSize(severity_text, font, font_scale, thickness)
    text_x = (panel3.shape[1] - text_width) // 2
    text_y = (panel3.shape[0] + text_height) // 2
    
    cv2.rectangle(panel3, (text_x - 10, text_y + 10), (text_x + text_width + 10, text_y - text_height - 15), (0,0,0), -1)
    cv2.putText(panel3, severity_text, (text_x, text_y), font, font_scale, (255, 255, 255), thickness)

    # Gabungkan ketiga panel
    # Resize semua panel ke tinggi yang sama sebelum menggabungkan
    target_height = max(panel1.shape[0], panel2.shape[0], panel3.shape[0])
    
    def resize_to_height(image, height):
        ratio = height / image.shape[0]
        width = int(image.shape[1] * ratio)
        return cv2.resize(image, (width, height), interpolation=cv2.INTER_AREA)

    panel1 = resize_to_height(panel1, target_height)
    panel2 = resize_to_height(panel2, target_height)
    panel3 = resize_to_height(panel3, target_height)

    combined_image = np.hstack((panel1, panel2, panel3))
    
    return combined_image


if __name__ == '__main__':
    os.makedirs(OUTPUT_FOLDER, exist_ok=True)
    
    print("Memuat model...")
    try:
        leaf_model = YOLO(LEAF_MODEL_PATH)
        disease_model = YOLO(DISEASE_MODEL_PATH)
        print("Model berhasil dimuat.")
    except Exception as e:
        print(f"Error memuat model: {e}")
        exit()
    
    image_files = [f for f in os.listdir(IMAGE_FOLDER) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

    if not image_files:
        print(f"Tidak ada gambar ditemukan di folder: {IMAGE_FOLDER}")
        exit()

    for filename in tqdm(image_files, desc="Menganalisis Gambar"):
        image_path = os.path.join(IMAGE_FOLDER, filename)
        
        # Langkah 1: Cari dan crop daun terbesar
        cropped_leaf_image = find_and_crop_largest_leaf(leaf_model, image_path)
        
        if cropped_leaf_image is None:
            print(f"\nTidak ada daun terdeteksi di {filename}. Dilewati.")
            continue
        
        # Langkah 2: Analisis gambar yang sudah di-crop
        severity, leaf_masks, disease_masks = analyze_cropped_image(leaf_model, disease_model, cropped_leaf_image)
        
        # Langkah 3: Buat visualisasi gabungan dari hasil crop
        final_visualization = create_combined_visualization(cropped_leaf_image, leaf_masks, disease_masks, severity)
        
        # Langkah 4: Simpan hasil akhir
        output_path = os.path.join(OUTPUT_FOLDER, filename)
        cv2.imwrite(output_path, final_visualization)

    print(f"\nSelesai! Semua hasil analisis disimpan di folder: {OUTPUT_FOLDER}")

Memuat model...
Model berhasil dimuat.


Menganalisis Gambar: 100%|██████████| 62/62 [00:12<00:00,  4.98it/s]


Selesai! Semua hasil analisis disimpan di folder: ../view/maskLesi/
